# Dynamic Time Warping measure

After observing the forecast in notebook `02-LSTM-experiment`, the delayed forecast can clearly be observed in the visualized storms, however, the used metrics do not show the model to give bad performance. We introduce a new measure based on dynamic time warping to detect this kind of error.

In [1]:
import pandas as pd
import numpy as np
%cd ..

from src.dtw.dtw_measure import dtw_measure

/c/Users/u0124144/Documents/DTW_measure


In [2]:
import h5py
# Import the data
def load_testing_sets(fname='data/processed/datasets.h5'):
    with h5py.File(fname, 'r') as f:
        test_in = f['test_sets/test_in'][:]
        test_out = f['test_sets/test_out'][:]
        predict = f['test_sets/prediction'][:]
        lookup = f['test_sets/lookup'][:]
    return test_in, test_out, predict, lookup.astype('datetime64[s]')

test_in, test_out, predict, lookup = load_testing_sets()
time_forward = 6

An important condition for DTW is that each time series is continuous, e.g. combining independent time series into one and evaluating this will give incorrect results. In notebook `01-data-preparation`, invalid measurements were removed, breaking the test data into a set of continous time series. All of these series must first be identified.

In [3]:
def extract_continuous_intervals(table):
    r'''Check lookup table for time discontinuities
    output: 
        Returns list of continouos times inside the lookup table
    '''
    lookup = pd.DataFrame(data=np.arange(table.shape[0]), index=pd.to_datetime(table[:,0]))
    lookup.index = pd.DatetimeIndex(lookup.index)
    # split = [g for n,g in lookup.groupby(pd.Grouper(freq='M')) if g.shape[0] != 0]

    min_size = 10
    timeseries = []
    
    #for month in split:
    series = lookup.index
    while len(series) > 0:
        # We can assume that the series starts from non-missing values, so the first diff gives sizes of continous intervals
        diff = pd.date_range(series[0], series[-1], freq='H').difference(series)
        if len(diff) > 0:
            if pd.Timedelta(diff[0] - pd.Timedelta('1h') - series[0])/pd.Timedelta('1h') > min_size:
                v1 = lookup.loc[series[0]][0]
                v2 = lookup.loc[diff[0] - pd.Timedelta('1h')][0]
                timeseries.append([v1, v2])
            if pd.Timedelta(series[-1] - diff[-1] - pd.Timedelta('1h'))/pd.Timedelta('1h') > min_size:
                v1 = lookup.loc[diff[-1] + pd.Timedelta('1h')][0]
                v2 = lookup.loc[series[-1]][0]
                timeseries.append([v1, v2])
            diff = pd.date_range(diff[0], diff[-1], freq='H').difference(diff)
        else:
            # Only when diff is empty
            v1 = lookup.loc[series[0]][0]
            v2 = lookup.loc[series[-1]][0]
            timeseries.append([v1, v2])
        series = diff
        

    return np.array(timeseries)
intervals = extract_continuous_intervals(lookup)

Now that we have continous intervals, the dtw measure is applied to each interval. From the resulting path, we measure the time shift between the mapping. The total counts are summarized in a pandas DataFrame, which is then normalized with `reformat_dtw_res` over the rows to provide a percentage. 

In [4]:
bincounts = np.zeros((time_forward,7))
counter = 0
for start, stop in intervals:
    counter += 1
    for i in range(time_forward):
        _, path, _ = dtw_measure(predict[start:stop, 0, i], test_out[start:stop, 0, i], time_forward)
        bins, counts = np.unique(abs(path[0, :] - path[1, :]), return_counts=True)
        bincounts[i, bins] += counts
        
lat_res = pd.DataFrame(data=bincounts, index=np.arange(1, time_forward+1), columns=np.arange(7))
print(lat_res)

         0        1        2        3        4        5        6
1  11602.0  21779.0   1483.0    443.0    171.0     80.0     40.0
2   4371.0  12677.0  16096.0   2772.0   1211.0    593.0    321.0
3   2950.0   4561.0  11184.0  13628.0   3921.0   2039.0   1465.0
4   2883.0   2619.0   4566.0   9661.0  12019.0   4700.0   3793.0
5   3166.0   2272.0   2980.0   4627.0   8654.0  10680.0   7581.0
6   3253.0   2217.0   2584.0   3261.0   4793.0   8048.0  15078.0


In [5]:
def reformat_dtw_res(df, filename=None):
    '''Normalize the result from the dtw measure
    '''
    res = df.div(df.sum(axis=1), axis=0)

    shifts = np.array(['t+{}h'.format(i+1) for i in np.arange(res.shape[0])])
    res['Prediction'] = shifts.T
    res = res.set_index('Prediction')
    res.columns = ['{}h'.format(i) for i in res.columns]
    res = res.apply(lambda x: round(x, 3))
    if filename:
        res.to_csv('{}reformated_{}'.format(path, filename))
    return res
        
reformat_dtw_res(lat_res)

,0h,1h,2h,3h,4h,5h,6h
Prediction,,,,,,,
t+1h,0.300,0.640,0.040,0.012,0.005,0.002,0.001
t+2h,0.106,0.324,0.442,0.074,0.033,0.014,0.008
t+3h,0.071,0.106,0.273,0.359,0.102,0.052,0.037
t+4h,0.070,0.065,0.107,0.227,0.309,0.122,0.099
t+5h,0.084,0.058,0.070,0.105,0.201,0.273,0.209
t+6h,0.093,0.059,0.066,0.079,0.109,0.191,0.402
